## HW2 Solution Part 1

**step1**: get keypoints for each images

**step2**: get patches for each keypoints

**step3**: get descriptions for each keypoints (by forward pass patches to the network)

step4: caculate the similarity matrices

step5: get topK similar images for each query

step6: draw recall vs precision curves

---

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [11]:
# load packages
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
#from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment

In [3]:
# parameters setting
stero_images_path = "stereo"
#image_path = "image_retrieval/images/"
#query_num = 35
image_num = 10
#kps_num = 30
patch_size = 32
patches = torch.zeros(query_num+image_num, kps_num,1, 32, 32)

In [4]:
# Initiate SURF detector
surf = cv2.xfeatures2d.SURF_create(30)

In [5]:
def getPatches(kps, img, size=32, num=500):
    res = torch.zeros(num, 1, size, size)
    if type(img) is np.ndarray:
        img = torch.from_numpy(img)
    h, w = img.shape      # note: for image, the x direction is the verticle, y-direction is the horizontal...
    for i in range(num):
        cx, cy = kps[i]
        cx, cy = int(cx), int(cy)
        dd = int(size/2)
        xmin, xmax = max(0, cx - dd), min(w, cx + dd ) - 1
        ymin, ymax = max(0, cy - dd), min(h, cy + dd ) - 1 
        
        xmin_res, xmax_res = dd - min(dd,cx), dd + min(dd, w - cx)-1
        ymin_res, ymax_res = dd - min(dd,cy), dd + min(dd, h - cy)-1

        res[i, 0, ymin_res: ymax_res, xmin_res: xmax_res] = img[ymin: ymax, xmin: xmax]
    return res

In [6]:
# tensor for query/image patches
for idx in range(query_num+image_num):
    if idx < query_num:
        img_dir = os.path.join(query_path,"q{}.JPG".format(idx+1))
    else:
        img_dir = os.path.join(image_path,"{}.JPG".format(idx+1-query_num))
    image = cv2.imread(img_dir)
    img= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ## find the keypoints and descriptors with SURF
    kps, des = surf.detectAndCompute(img, None)
    ## find the keypoints and descriptors with SIFT
    keypoints_img = [kps[a].pt for a in range(kps_num)] 
    patches[idx] = getPatches(keypoints_img, img, size=patch_size, num=kps_num)    

In [7]:
# import network, , load pretrained weights, and turn on testing mode
from descriptor_CNN3 import DesNet
model = DesNet()
model.cuda()
model.eval()
trained_weights = torch.load("checkpoint.pth")
model.load_state_dict(trained_weights["state_dict"])

In [8]:
print(patches.shape)

torch.Size([175, 30, 1, 32, 32])


In [9]:
# get descriptions
with torch.no_grad(): # only 3612MiB, without this line, it will be 9132MiB
    des1 = model(patches[:query_num].view(-1, 1, 32,32).cuda()).view(-1, 30, 128)
    des2 = model(patches[query_num:].view(-1, 1, 32,32).cuda()).view(-1, 30, 128)

In [10]:
print(des1.shape, des2.shape)
# save des
des_dir = "des.pt"
torch.save([des1, des2], des_dir)

torch.Size([35, 30, 128]) torch.Size([140, 30, 128])


In [97]:
np.set_printoptions(suppress=True)
# Image 11 and 12
x11 = [[143, 232, 1],[16, 400, 1],[59, 480, 1],[167, 500, 1],[363, 569, 1],[597, 411, 1],
[447, 641, 1],[879, 50, 1], [781, 528, 1],[779, 636, 1]]

x21 = [[179, 236, 1], [59, 400, 1], [97, 480, 1], [195, 498, 1], [380, 571, 1], [609, 413, 1],
[461, 643, 1], [892, 49, 1], [782, 530, 1], [779, 638, 1]]

W = np.zeros((10, 9))

#for m in range(0, 10):
for i in range(0, 10):
    W[i, 0]= x11[i][0]*x21[i][0]
    W[i, 1]= x11[i][0]*x21[i][1]
    W[i, 2]= x11[i][0]
    W[i, 3]= x11[i][1]*x21[i][0]
    W[i, 4]= x11[i][1]*x21[i][1]
    W[i, 5]= x11[i][1]
    W[i, 6]= x21[i][0]
    W[i, 7]= x21[i][1]
    W[i,8]=1

print("W matrix: W F = 0\n")
print(W)

print('____________________________________________________________________________________')

# SVD
print("\n SVD ... \n")

u, s, vh = np.linalg.svd(W, full_matrices=True)
# last row of V = last column V tranposed

f = vh[-1]
print("f = last column V tranposed \n")
print(f)

F = np.zeros((3,3))

for i in range(0,3):
    for j in range(0,3):
        F[i,j]=f[i*3+j]

print("Fundamental Matrix\n")
print(F)


W matrix: W F = 0

[[ 25597.  33748.    143.  41528.  54752.    232.    179.    236.      1.]
 [   944.   6400.     16.  23600. 160000.    400.     59.    400.      1.]
 [  5723.  28320.     59.  46560. 230400.    480.     97.    480.      1.]
 [ 32565.  83166.    167.  97500. 249000.    500.    195.    498.      1.]
 [137940. 207273.    363. 216220. 324899.    569.    380.    571.      1.]
 [363573. 246561.    597. 250299. 169743.    411.    609.    413.      1.]
 [206067. 287421.    447. 295501. 412163.    641.    461.    643.      1.]
 [784068.  43071.    879.  44600.   2450.     50.    892.     49.      1.]
 [610742. 413930.    781. 412896. 279840.    528.    782.    530.      1.]
 [606841. 497002.    779. 495444. 405768.    636.    779.    638.      1.]]
____________________________________________________________________________________

 SVD ... 

f = last column V tranposed 

[-0.00000012  0.00004452 -0.02110133 -0.00004641 -0.00000091  0.01201702
  0.02204132 -0.00950205 -0.99